In [28]:
import pandas as pd
import numpy as np
import rpy2.robjects as robjects
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
import time
import os
import csv
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

sns.set_style("white")

pandas2ri.activate()

from PyCROSL import SubstrateReal, AbsObjectiveFunc, CRO_SL
import random
from objective_functions import *

rscript_name_single = "exec_optim.R"
rscript_name_cascade = "exec_optim_semidist.R"

In [29]:
basin_n = 3005  # 3005 or 5043

if basin_n == 3005:
    data_file_single = "data/data_CHT_3005.txt"
    data_file_cascade = "data/data_CHT_SIMPA.txt"
    basin_file_single = "data/basins_CHT.txt"
    basin_file_cascade = "data/basins_CHT.txt"
elif basin_n == 5043:
    data_file_single = "data/data_CHG_5043.txt"
    data_file_cascade = "data/data_CHG_SIMPA.txt"
    basin_file_single = "data/basins_CHG_5043.txt"
    basin_file_cascade = "data/basins_CHG.txt"
else:
    raise Exception("Invalid basin.")

In [31]:
# Defining the R script and loading the instance in Python
r = robjects.r
r["source"](rscript_name_single)
r["source"](rscript_name_cascade)

robjects.globalenv["init_global"](data_file_cascade, basin_file_cascade)
exec_function_r = robjects.globalenv["eval_basin_param"]
get_basin_q = robjects.globalenv["get_basin_q"]


weights = []
basin_df = pd.read_csv(basin_file_cascade)
basin_df.sort_values(by=["order"])

full_data = pd.read_csv(data_file_cascade)
for i, idx in enumerate(basin_df.index):
    basin_code = basin_df["code"][idx]
    total_caudal = full_data[full_data["qhmobs"] != -100][full_data["code"] == basin_code]["qhmobs"].sum()
    weights.append(total_caudal)
print(weights)
weights = np.asarray(weights) / sum(weights)
print(weights)

Rows: 4 Columns: 4
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (4): code, order, codedown, supha

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
[3417.6547, 1270.3415, 809.4765, 11512.4452]
[0.20092129 0.0746824  0.0475885  0.67680781]


In [32]:
# result_path = "results/new_data/"
result_path = ""
exec_types = [
    ("fullpon", "KGE", 1),
    ("fullpon", "MSE", 0),
    ("full", "NSE", 1),
]

result_files = [(f"./{result_path}config_{problem_type}_{basin_n}_{model}_{metric}.csv", problem_type, model, metric) for problem_type, metric, model in exec_types]

for file_name, *_ in result_files:
    print(file_name, "Yes" if os.path.exists(file_name) else "No")
    if not os.path.exists(file_name):
        raise Exception("Some files were not found.")

./config_fullpon_3005_1_KGE.csv Yes
./config_fullpon_3005_0_MSE.csv Yes
./config_full_3005_1_NSE.csv Yes


In [33]:
print(np.loadtxt(result_files[0][0], delimiter=",", max_rows=1).shape)

(28,)


In [34]:
eval_df = pd.DataFrame(columns=["model", "target", "MSE", "RMSE", "Pbias", "NSE", "R2", "KGE", "params"])
params_list = []
for file_name, problem_type, model, target in result_files:
    print(file_name, "Yes" if os.path.exists(file_name) else "No")
    if problem_type == "single":
        robjects.globalenv["init_global"](data_file, basin_file)
        params = np.loadtxt(file_name, delimiter=",", max_rows=1)
        params_list.append(",".join(map(str, params)))

        metrics = exec_function_r(model, params)
        param_str = np.array2string(params, max_line_width=np.inf, separator=";").replace(" ", "")
        eval_df.loc[len(eval_df)] = [model, target] + list(metrics) + [param_str]
    elif problem_type in ["full", "fullpon"]:
        params = np.loadtxt(file_name, delimiter=",", max_rows=1).reshape([len(basin_df), -1])

        prev_q = 0
        agg_q = {}
        basin_params = {}
        for idx in basin_df.index:
            basin_code = basin_df["code"][idx]
            codedown = basin_df["codedown"][idx]

            prev_q = 0
            if basin_code in agg_q:
                prev_q = agg_q[basin_code]

            if codedown not in agg_q:
                agg_q[codedown] = 0

            agg_q[codedown] += get_basin_q(model, params[idx], basin_code, prev_q)

            metrics = exec_function_r(model, params[idx], basin_code, prev_q)
            param_str = np.array2string(params[idx], max_line_width=np.inf, separator=";").replace(" ", "")
            eval_df.loc[len(eval_df)] = [model, target, basin_code] + list(metrics) + [param_str]
    

eval_df = eval_df[eval_df["target"] != "R2"]
eval_df

./config_fullpon_3005_1_KGE.csv Yes


ValueError: cannot set a row with mismatched columns

In [20]:
eval_df.sort_values("NSE")

,model,target,MSE,RMSE,Pbias,NSE,R2,KGE,params
2,1,KGE,120.990025,10.999547,-1.8,0.414461,0.507805,0.711547,[0.6950017;372.13525704;1.;0.48689489;0.503093...
0,1,NSE,109.551104,10.466666,-3.6,0.469821,0.482026,0.628063,[1.71690187e-01;4.71663208e+02;9.99999982e-01;...
1,1,MSE,109.551080,10.466665,-3.6,0.469821,0.481995,0.627890,[1.70416486e-01;4.71184997e+02;9.99999997e-01;...


In [41]:
# code,order,codedown,supha,MSE,RMSE,PBIAS,NSE,R2,KGE,f0,param
eval_df["code"] = 5043
eval_df["order"] = 1
eval_df["codedown"] = 0
eval_df["supha"] = 329429.24
eval_df["f0"] = -100
eval_df["PBIAS"] = -100
eval_table_df = eval_df[["code", "order", "codedown", "supha", "MSE", "RMSE", "PBIAS", "NSE", "R2", "KGE", "f0", "params"]]
eval_table_df.loc[:, "params"] = eval_table_df.loc[:, "params"].str.replace("[", "")
eval_table_df.loc[:, "params"] = eval_table_df.loc[:, "params"].str.replace("]", "")
eval_table_df

,code,order,codedown,supha,MSE,RMSE,PBIAS,NSE,R2,KGE,f0,params
0,5043,1,0,329429.24,109.551104,10.466666,-100,0.469821,0.482026,0.628063,-100,1.71690187e-01;4.71663208e+02;9.99999982e-01;4...
1,5043,1,0,329429.24,109.551080,10.466665,-100,0.469821,0.481995,0.627890,-100,1.70416486e-01;4.71184997e+02;9.99999997e-01;4...
2,5043,1,0,329429.24,120.990025,10.999547,-100,0.414461,0.507805,0.711547,-100,0.6950017;372.13525704;1.;0.48689489;0.5030936...


In [39]:
eval_table_df.to_csv(f"table_{problem_type}.txt", index=False, quoting=csv.QUOTE_NONE)

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(20, 5))


metric_measures = ["NSE", "MSE", "Pbias", "KGE"]
for idx, metric in enumerate(["NSE", "MSE", "Pbias", "KGE"]):
    # ax = axes[idx//2, idx%2]
    ax = axes[idx]
    # hist_ax = sns.barplot(data=data_to_plot, x="target", y=metric_measure, hue="model", ax=ax)
    hist_ax = sns.barplot(data=eval_df, x="target", y=metric_measures[idx], hue="model", ax=ax)
    hist_ax.set(title=f"Measuring {metric}")

    if metric == "NSE":
        ax.axhline(1, color="k", linestyle="--")
        hist_ax.set(ylim=(0, 1.1))
    elif metric_measures[idx] == "Pbias":
        hist_ax.set(ylim=(-20, 20))
        pass
    elif metric != "MSE":
        ax.axhline(1, color="k", linestyle="--")
        hist_ax.set(ylim=(0, 1.1))

    if idx == 3:
        sns.move_legend(hist_ax, "upper left", bbox_to_anchor=(1, 1))
    else:
        hist_ax.get_legend().remove()
    sns.despine()
    ax.grid(axis="y")

plt.show()